In [ ]:
import os
import numpy as np
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
import statistics
from scipy.stats import weibull_min,weibull_max
from sklearn import preprocessing
import tensorflow as tf
from tensorflow.keras import initializers, regularizers
from tensorflow.keras import backend as k
from tensorflow.keras.layers import Input, multiply,Layer
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import RMSprop,SGD,Adam
from tensorflow.keras.models import Model
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test
from lifelines.utils import concordance_index

In [ ]:
#input 5 5fold image features
stage12_fold1=pd.read_pickle(path_to_fold1_feature_pickle)
stage12_fold2=pd.read_pickle(path_to_fold2_feature_pickle)
stage12_fold3=pd.read_pickle(path_to_fold3_feature_pickle)
stage12_fold4=pd.read_pickle(path_to_fold4_feature_pickle)
stage12_fold5=pd.read_pickle(path_to_fold5_feature_pickle)

In [ ]:
train_df = pd.concat([stage12_fold1,stage12_fold2,stage12_fold3,stage12_fold4])
test_df = stage12_fold5
train_df=train_df.reset_index(drop=True)
test_df=test_df.reset_index(drop=True)

train_df.to_pickle('train_df.pkl')
test_df.to_pickle('test_df.pkl')

In [ ]:
class Mil_Attention(Layer):
    """
    # copy from https://github.com/utayao/Atten_Deep_MIL
    Mil Attention Mechanism
    This layer contains Mil Attention Mechanism
    # Input Shape
        2D tensor with shape: (batch_size, input_dim)
    # Output Shape
        2D tensor with shape: (1, units)
    """

    def __init__(self, L_dim, output_dim, kernel_initializer='glorot_uniform', kernel_regularizer=None,
                    use_bias=True, use_gated=False, **kwargs):
        self.L_dim = L_dim
        self.output_dim = output_dim
        self.use_bias = use_bias
        self.use_gated = use_gated

        self.v_init = initializers.get(kernel_initializer)
        self.w_init = initializers.get(kernel_initializer)
        self.u_init = initializers.get(kernel_initializer)


        self.v_regularizer = regularizers.get(kernel_regularizer)
        self.w_regularizer = regularizers.get(kernel_regularizer)
        self.u_regularizer = regularizers.get(kernel_regularizer)

        super(Mil_Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 2
        input_dim = input_shape[1]

        self.V = self.add_weight(shape=(input_dim, self.L_dim),
                                      initializer=self.v_init,
                                      name='v',
                                      regularizer=self.v_regularizer,
                                      trainable=True)


        self.w = self.add_weight(shape=(self.L_dim, 1),
                                    initializer=self.w_init,
                                    name='w',
                                    regularizer=self.w_regularizer,
                                    trainable=True)


        if self.use_gated:
            self.U = self.add_weight(shape=(input_dim, self.L_dim),
                                     initializer=self.u_init,
                                     name='U',
                                     regularizer=self.u_regularizer,
                                     trainable=True)
        else:
            self.U = None

        self.input_built = True


    def call(self, x, mask=None):
        n, d = x.shape
        ori_x = x
        # do Vhk^T
        x = k.tanh(k.dot(x, self.V)) # (2,64)

        if self.use_gated: #no gate
            gate_x = k.sigmoid(k.dot(ori_x, self.U))
            ac_x = x * gate_x
        else:
            ac_x = x

        # do w^T x
        soft_x = k.dot(ac_x, self.w)  # (2,64) * (64, 1) = (2,1)
        alpha = k.softmax(k.transpose(soft_x)) # (2,1)  #change
        alpha = k.transpose(alpha)
        return alpha

    def compute_output_shape(self, input_shape):
        shape = list(input_shape)
        assert len(shape) == 2
        shape[1] = self.output_dim
        return tuple(shape)

    def get_config(self):
        config = {
            'output_dim': self.output_dim,
            'v_initializer': initializers.serialize(self.V.initializer),
            'w_initializer': initializers.serialize(self.w.initializer),
            'v_regularizer': regularizers.serialize(self.v_regularizer),
            'w_regularizer': regularizers.serialize(self.w_regularizer),
            'use_bias': self.use_bias
        }
        base_config = super(Mil_Attention, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


class Last_Sigmoid(Layer):
    """
    Attention Activation
    This layer contains a FC layer which only has one neural with sigmoid actiavtion
    and MIL pooling. The input of this layer is instance features. Then we obtain
    instance scores via this FC layer. And use MIL pooling to aggregate instance scores
    into bag score that is the output of Score pooling layer.
    This layer is used in mi-Net.
    # Arguments
        output_dim: Positive integer, dimensionality of the output space
        kernel_initializer: Initializer of the `kernel` weights matrix
        bias_initializer: Initializer of the `bias` weights
        kernel_regularizer: Regularizer function applied to the `kernel` weights matrix
        bias_regularizer: Regularizer function applied to the `bias` weights
        use_bias: Boolean, whether use bias or not
        pooling_mode: A string,
                      the mode of MIL pooling method, like 'max' (max pooling),
                      'ave' (average pooling), 'lse' (log-sum-exp pooling)
    # Input shape
        2D tensor with shape: (batch_size, input_dim)
    # Output shape
        2D tensor with shape: (1, units)
    """
    def __init__(self, output_dim, kernel_initializer='glorot_uniform', bias_initializer='zeros',
                    kernel_regularizer=None, bias_regularizer=None,
                    use_bias=True, **kwargs):
        self.output_dim = output_dim

        self.kernel_initializer = initializers.get(kernel_initializer)
        self.bias_initializer = initializers.get(bias_initializer)
        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)

        self.use_bias = use_bias
        super(Last_Sigmoid, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 2
        input_dim = input_shape[1]

        self.kernel = self.add_weight(shape=(input_dim, self.output_dim),
                                        initializer=self.kernel_initializer,
                                        name='kernel',
                                        regularizer=self.kernel_regularizer)

        if self.use_bias:
            self.bias = self.add_weight(shape=(self.output_dim,),
                                        initializer=self.bias_initializer,
                                        name='bias',
                                        regularizer=self.bias_regularizer)
        else:
            self.bias = None

        self.input_built = True
    
    def activate(ab):
        a = k.exp(ab[:, 0])
        b = k.softplus(ab[:, 1])
        a = k.reshape(a, (k.shape(a)[0], 1))
        b = k.reshape(b, (k.shape(b)[0], 1))
        return k.concatenate((a, b), axis=1)

    def call(self, x, mask=None):
        n, d = x.shape
        x = k.sum(x, axis=0, keepdims=True)
        # compute instance-level score
        x = k.dot(x, self.kernel)
        if self.use_bias:
            x = k.bias_add(x, self.bias)

        # sigmoid
        out = activate(x) #change


        return out

    def compute_output_shape(self, input_shape):
        shape = list(input_shape)
        assert len(shape) == 2
        shape[1] = self.output_dim
        return tuple(shape)

    def get_config(self):
        config = {
            'output_dim': self.output_dim,
            'kernel_initializer': initializers.serialize(self.kernel.initializer),
            'bias_initializer': initializers.serialize(self.bias_initializer),
            'kernel_regularizer': regularizers.serialize(self.kernel_regularizer),
            'bias_regularizer': regularizers.serialize(self.bias_regularizer),
            'use_bias': self.use_bias
        }
        base_config = super(Last_Sigmoid, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'

    

    
    def __init__(self, patient_bar, labels,status, batch_size=32, dim=(299,299), n_channels=3,
                 folder='train',shuffle=False):
        
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.status = status
        self.patient_bar=patient_bar

        self.n_channels = n_channels
        self.shuffle = shuffle
        self.folder = folder
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.patient_bar) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        
        # Find list of IDs
        patient_bar_temp = [self.patient_bar[k] for k in indexes]

        # Generate data
        X_person, y  = self.__data_generation(indexes)
        
       
        return X_person, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.patient_bar))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, patient_bar_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)

        train_df = pd.read_pickle('train_df.pkl')
        test_df = pd.read_pickle('test_df.pkl')
                
        
        real_day=list(train_df['Days'])
        int_day = map(int, real_day)
        real_day=list(int_day)
        day_mean=statistics.mean(real_day)
        day_std=statistics.stdev(real_day)
        Days_std=[((i -day_mean)/day_std)+n for i in real_day] #please input n (make sure Days_std >0)

        train_df['Days_std'] = Days_std
        train_df=train_df.reset_index(drop=True)

        test_day=list(test_df['Days'])
        int_test = list(map(int, test_day))
        Days_std_test=[((i -day_mean)/day_std)+n for i in int_test] #please input n

        test_df['Days_std'] = Days_std_test
        test_df=test_df.reset_index(drop=True)


        feature_columns=train_df.columns[xi:xj]  #select feature columns
        
        X_person = []
        
        y=[]
        y_d = []
        y_s = []

        # Generate data
        for i, ID in enumerate(patient_bar_temp):
            train_array=[]
            
            if self.folder == 'train_df':
                per_person=train_df.loc[train_df['bcr_patient_barcode'] == self.patient_bar[ID]] #select a person
            
            else :
                per_person=test_df.loc[test_df['bcr_patient_barcode'] == self.patient_bar[ID]] #select a person
                
            per_person_array=(per_person[feature_columns]).to_numpy() #per person features to array
            train_array.append(per_person_array)
            
            X_person.append(train_array)
            

            
            y_d.append(list(per_person['Days'])[0])
            y_s.append(list(per_person['vital_status'])[0])

            
            
        X_person = np.array(X_person).squeeze()
        
              
        y_d = np.array(y_d).astype('float')[:, np.newaxis]
        
        y_s = np.array(y_s).astype('int')[:, np.newaxis]
        y_temp = np.hstack((y_d, y_s))   
        
        y = tf.convert_to_tensor(y_temp, dtype=tf.float32)
        
        return X_person, y

In [ ]:
# n:lambda  a:kappa
def weib(x,a,n):
    return (a / n) * (x / n)**(a - 1) * np.exp(-(x / n)**a)

In [ ]:
#Loss function

def weibull_loglik_discrete(y_true, ab_pred, name=None):
    y_ = y_true[:, 0]
    u_ = y_true[:, 1]
    a_ = ab_pred[:, 0]
    b_ = ab_pred[:, 1]
    hazard0 = k.pow((y_ + 1e-35) / a_, b_)
    hazard1 = k.pow((y_ + 1) / a_, b_)
    return -1 * k.mean(u_ * k.log(k.exp(hazard1 - hazard0) - 1.0) - hazard1)


def weibull_loglik_continuous(y_true, ab_pred, name=None):
    y_ = y_true[:, 0]
    u_ = y_true[:, 1]
    a_ = ab_pred[:, 0]
    b_ = ab_pred[:, 1]  # death / live
    ya = (y_ + 1e-35) / a_
    return -1 * k.mean(u_ * (k.log(b_) + b_ * k.log(ya)) - k.pow(ya, b_))


def activate(ab):
    a = k.exp(ab[:, 0])
    b = k.softplus(ab[:, 1])
    a = k.reshape(a, (k.shape(a)[0], 1))
    b = k.reshape(b, (k.shape(b)[0], 1))
    return k.concatenate((a, b), axis=1)

In [ ]:
train_labels = train_df['Days_std']
train_status = train_df['vital_status']
train_bar= list(set(train_df['bcr_patient_barcode']))

test_labels = test_df['Days_std']
test_status = test_df['vital_status']
test_bar= list(set(test_df['bcr_patient_barcode']))

params_train = {'dim': (299,299),'batch_size': 1,'n_channels': 3,'shuffle': False,'folder':'train_df'}
params_test = {'dim': (299,299),'batch_size': 1,'n_channels': 3,'shuffle': False,'folder':'test_df'}

In [ ]:
# Generators
training_generator = DataGenerator(train_bar, train_labels,train_status, **params_train )
validation_generator = DataGenerator(test_bar, test_labels,test_status, **params_test)

## Model

In [ ]:
data_input = Input(shape=(2048), dtype='float32', name='input')
alpha = Mil_Attention(L_dim=2048, output_dim=2, kernel_regularizer=l2(0.000001), name='alpha', use_gated='False')(data_input)
x_mul = multiply([alpha, data_input])
out = Last_Sigmoid(output_dim=2, name='FC1_sigmoid')(x_mul)
x_out = out
model = Model(inputs=data_input, outputs=x_out)

# for layer in base_model.layers:
#     layer.trainable = False

In [ ]:
model.compile(loss=loss_function,  optimizer=optimizer) #please input loss function,optimizer

In [ ]:
model.fit(training_generator,
          validation_data=validation_generator,
          epochs=epochs,  #please input epochs
          )

## Evaluation

In [ ]:
probabilities = model.predict(training_generator)

In [ ]:
data = {'bcr_patient_barcode':  train_bar}
infer_train_df = pd.DataFrame (data, columns = ['bcr_patient_barcode'])

train_origin=train_df[['bcr_patient_barcode','Days','Days_std','vital_status']].drop_duplicates(subset=['bcr_patient_barcode'], keep='last')
infer_train_df = pd.merge(infer_train_df, train_origin, on=['bcr_patient_barcode'])

In [ ]:

train_predict = np.resize(probabilities, (len(infer_train_df) , 2))



train_result = np.concatenate((np.array(infer_train_df['Days'])[:, np.newaxis], train_predict), axis=1)

predict_list=[]
kappa_list=[]
lambda_list=[]
for i in range(0,len(train_result)):
#pdf peak
#     x = np.arange(0,15,0.0001)
#     per_list=weibull_min.pdf(x, scale=train_result[i][1], c=train_result[i][2])
#     per=x[np.argmax(per_list)]
    
#cdf 0.5
    per=weibull_min.ppf(0.5, scale=train_result[i][1], c=train_result[i][2])

#pdf monte carlo 10000 - y
#     x = [random.uniform(0, 15) for i in range(20000)]
#     per_list=weibull_min.pdf(x, scale=train_result[i][1], c=train_result[i][2])
#     #new_list = np.append(per_list, per)
#     per=statistics.mean(per_list)

#pdf monte carlo 10000 - x
#     x = np.arange(0,15,0.0001)
#     per_list=weibull_min.pdf(x, scale=train_result[i][1], c=train_result[i][2])
#     per_list=per_list/sum(per_list)
#     x_list = np.random.choice(x,size=10000, p = per_list.ravel())
# #     per=statistics.mean(per_list)
#     per=stats.mode(x_list)[0][0]
    
    
    kappa=train_result[i][2]
    lamda=train_result[i][1]
    kappa_list.append(kappa)
    lambda_list.append(lamda)
    predict_list.append(per)

infer_train_df['predict_std_day']=predict_list
infer_train_df['lambda']=lambda_list
infer_train_df['kappa']=kappa_list


#C-index
print(concordance_index(infer_train_df['Days'], infer_train_df['predict_std_day'], infer_train_df['vital_status']))

In [ ]:
probabilities_test = model.predict(validation_generator)

In [ ]:
data_test = {'bcr_patient_barcode':  test_bar}
infer_test_df = pd.DataFrame (data_test, columns = ['bcr_patient_barcode'])

test_origin=test_df[['bcr_patient_barcode','Days','Days_std','vital_status']].drop_duplicates(subset=['bcr_patient_barcode'], keep='last')
infer_test_df = pd.merge(infer_test_df, test_origin, on=['bcr_patient_barcode'])

In [ ]:
test_predict = np.resize(probabilities_test, (len(infer_test_df) , 2))
test_result = np.concatenate((np.array(infer_test_df['Days'])[:, np.newaxis], test_predict), axis=1)

predict_list=[]
kappa_list=[]
lambda_list=[]
for i in range(0,len(test_result)):
#pdf peak
#     x = np.arange(0,15,0.0001)
#     per_list=weibull_min.pdf(x, scale=train_result[i][1], c=train_result[i][2])
#     per=x[np.argmax(per_list)]
    
#cdf 0.5
    per=weibull_min.ppf(0.5, scale=test_result[i][1], c=test_result[i][2])

#pdf monte carlo 10000 - y
#     x = [random.uniform(0, 15) for i in range(20000)]
#     per_list=weibull_min.pdf(x, scale=train_result[i][1], c=train_result[i][2])
#     #new_list = np.append(per_list, per)
#     per=statistics.mean(per_list)

#pdf monte carlo 10000 - x
#     x = np.arange(0,15,0.0001)
#     per_list=weibull_min.pdf(x, scale=train_result[i][1], c=train_result[i][2])
#     per_list=per_list/sum(per_list)
#     x_list = np.random.choice(x,size=10000, p = per_list.ravel())
# #     per=statistics.mean(per_list)
#     per=stats.mode(x_list)[0][0]
    
    
    kappa=test_result[i][2]
    lamda=test_result[i][1]
    kappa_list.append(kappa)
    lambda_list.append(lamda)
    predict_list.append(per)

infer_test_df['predict_std_day']=predict_list
infer_test_df['lambda']=lambda_list
infer_test_df['kappa']=kappa_list

#C-index
print(concordance_index(infer_test_df['Days'], infer_test_df['predict_std_day'], infer_test_df['vital_status']))

## statistics and kmcurve

In [ ]:
infer_train_df['Days'] = [int(i) for i in infer_train_df['Days']]
med_day=np.median(list(infer_train_df['predict_std_day']))
med_day

In [ ]:
# infer_test_df['Days'] = [int(i) for i in infer_test_df['Days']]
# med_day=np.median(list(infer_test_df['predict_std_day']))
# med_day

In [ ]:
infer_test_df["group"]=''

for i in range(0,len(infer_test_df)):
    if infer_test_df['predict_std_day'][i] > med_day :
        infer_test_df["group"][i]=1 #big
    else :
        infer_test_df["group"][i]=0 #small

In [ ]:
result=infer_test_df

T_big=[]
T_small=[]
E_big=[]
E_small=[]
for i in range(0,len(result)):
    if result['group'][i]==1: #big
        T_big.append(result['Overall Survival (Months)'][i])
        E_big.append(result['Overall Survival Status'][i])
    else: #small
        T_small.append(result['Overall Survival (Months)'][i])
        E_small.append(result['Overall Survival Status'][i])        

In [ ]:
logrank_results = logrank_test(T_big, T_small, event_observed_A=E_big, event_observed_B=E_small)
logrank_results

In [ ]:
kmf = KaplanMeierFitter()
ax = plt.subplot(111)

kmf.fit(T_small, event_observed=E_small, label="Shorter-term survivors")
kmf.plot(show_censors=True,ax=ax)

kmf.fit(T_big, event_observed=E_big, label="Longer-term survivors")
kmf.plot(show_censors=True,ax=ax)